In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/show/Inpaint-Anything

/content/drive/MyDrive/show/Inpaint-Anything


In [ ]:
#!git clone https://github.com/geekyutao/Inpaint-Anything.git

Cloning into 'Inpaint-Anything'...
remote: Enumerating objects: 2090, done.
remote: Counting objects: 100% (162/162), done.
remote: Compressing objects: 100% (79/79), done.
^C


In [ ]:
!pip install torch torchvision torchaudio
!pip install -e segment_anything
!pip install -r lama/requirements.txt

Obtaining file:///content/drive/MyDrive/show/Inpaint-Anything/segment_anything
  Preparing metadata (setup.py) ... done
  Attempting uninstall: segment-anything
    Found existing installation: segment-anything 1.0
    Uninstalling segment-anything-1.0:
      Successfully uninstalled segment-anything-1.0
  Running setup.py develop for segment-anything


In [ ]:
!pip uninstall -y torchtext

In [ ]:
!pip install torchtext==0.5.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 901.8 kB/s eta 0:00:00


In [ ]:
cd /content/drive/MyDrive/show/Inpaint-Anything

/content/drive/MyDrive/show/Inpaint-Anything


In [ ]:
import os

# Define the paths to the input image folder and output results folder
input_folder = "/content/drive/MyDrive/show/data/pisa_pre1"
output_folder = "/content/drive/MyDrive/show/data/output"

#Create the output results folder (if it does not exist)
os.makedirs(output_folder, exist_ok=True)

# Loop through each image
for filename in os.listdir(input_folder):
    # Build complete input and output file paths
    input_img_path = os.path.join(input_folder, filename)
    output_img_path = os.path.join(output_folder, filename)

    # Use !python remove_anything.py to process the current image
    !python remove_anything.py \
        --input_img "$input_img_path" \
        --coords_type key_in \
        --point_coords 360 180 \
        --point_labels 1 \
        --dilate_kernel_size 15 \
        --output_dir "$output_folder" \
        --sam_model_type "vit_h" \
        --sam_ckpt ./pretrained_models/sam_vit_h_4b8939.pth \
        --lama_config ./lama/configs/prediction/default.yaml \
        --lama_ckpt ./pretrained_models/big-lama

/content/drive/MyDrive/show/Inpaint-Anything/segment_anything/segment_anything/modeling/tiny_vit_sam.py:657: UserWarning: Overwriting tiny_vit_5m_224 in registry with segment_anything.modeling.tiny_vit_sam.tiny_vit_5m_224. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  return register_model(fn_wrapper)
/content/drive/MyDrive/show/Inpaint-Anything/segment_anything/segment_anything/modeling/tiny_vit_sam.py:657: UserWarning: Overwriting tiny_vit_11m_224 in registry with segment_anything.modeling.tiny_vit_sam.tiny_vit_11m_224. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  return register_model(fn_wrapper)
/content/drive/MyDrive/show/Inpaint-Anything/segment_anything/segment_anything/modeling/tiny_vit_sam.py:657: UserWarning: Overwriting tiny_vit_21m_224 in registry with segment_anything.modeling.tiny_vit_sam.tiny_vit_21m_224. This is because the name bein

In [ ]:
import cv2
import numpy as np

def is_color_similar(color1, color2, threshold=30):
    return np.linalg.norm(np.array(color1) - np.array(color2)) < threshold

def adjust_brightness_contrast(image, brightness=35, contrast=-45):
    """Adjust the brightness and contrast of the image"""
    if brightness != 0:
        if brightness > 0:
            shadow = brightness
            highlight = 255
        else:
            shadow = 0
            highlight = 255 + brightness
        alpha_b = (highlight - shadow) / 255
        gamma_b = shadow
        image = cv2.addWeighted(image, alpha_b, image, 0, gamma_b)

    if contrast != 0:
        f = 131 * (contrast + 127) / (127 * (131 - contrast))
        alpha_c = f
        gamma_c = 127 * (1 - f)
        image = cv2.addWeighted(image, alpha_c, image, 0, gamma_c)

    return image

def process_image_specific_color_range(image_path, output_path, target_color, color_threshold=30, brightness=35, contrast=-45):
    image = cv2.imread(image_path)

    # Find pixels close to the target color
    mask = np.zeros(image.shape[:2], dtype=np.uint8)
    for y in range(image.shape[0]):
        for x in range(image.shape[1]):
            if is_color_similar(image[y, x], target_color, color_threshold):
                mask[y, x] = 255

    # Adjust brightness and contrast only for pixels of the target color
    adjusted_image = image.copy()
    adjusted_image[mask == 255] = adjust_brightness_contrast(image[mask == 255], brightness, contrast)

    # save image
    cv2.imwrite(output_path, adjusted_image)



input_image_path = '/content/drive/MyDrive/show/data/pise_sha/frame_0001.jpg'
output_image_path = '/content/drive/MyDrive/test/test_001.jpg'
target_color = [15, 20, 16]

process_image_specific_color_range(input_image_path, output_image_path, target_color)

In [ ]:
import cv2
import numpy as np

def is_color_similar(color1, color2, threshold=30):
    return np.linalg.norm(np.array(color1) - np.array(color2)) < threshold

def adjust_brightness_contrast(image, brightness=60, contrast=-40):
    """Adjust the brightness and contrast of the image"""
    if brightness != 0:
        if brightness > 0:
            shadow = brightness
            highlight = 255
        else:
            shadow = 0
            highlight = 255 + brightness
        alpha_b = (highlight - shadow) / 255
        gamma_b = shadow
        image = cv2.addWeighted(image, alpha_b, image, 0, gamma_b)

    if contrast != 0:
        f = 131 * (contrast + 127) / (127 * (131 - contrast))
        alpha_c = f
        gamma_c = 127 * (1 - f)
        image = cv2.addWeighted(image, alpha_c, image, 0, gamma_c)

    return image

def feather_mask(mask, feather_amount=5):
    kernel_size = feather_amount * 2 + 1
    blurred_mask = cv2.GaussianBlur(mask, (kernel_size, kernel_size), 0)
    return blurred_mask

def blend_images(image1, image2, mask, alpha=0.5):
    blended = image1.copy()
    alpha_mask = mask.astype(float) / 255 * alpha
    for c in range(0, 3):
        blended[:, :, c] = (alpha_mask * image2[:, :, c] + (1 - alpha_mask) * image1[:, :, c])
    return blended

def process_image_specific_color_range_and_blend(input_image_path, output_image_path, target_color, color_threshold=30, brightness=35, contrast=-45, feather_amount=7, alpha=0.65):
    image = cv2.imread(input_image_path)

    # Find pixels close to the target color
    mask = np.zeros(image.shape[:2], dtype=np.uint8)
    for y in range(image.shape[0]):
        for x in range(image.shape[1]):
            if is_color_similar(image[y, x], target_color, color_threshold):
                mask[y, x] = 255

    # Adjust brightness and contrast only for pixels of the target color
    adjusted_image = image.copy()
    adjusted_image[mask == 255] = adjust_brightness_contrast(image[mask == 255], brightness, contrast)

    # Feather Mask
    feathered_mask = feather_mask(mask, feather_amount)

    # Overlay the adjusted image with the original image
    final_image = blend_images(image, adjusted_image, feathered_mask, alpha)

    # save image
    cv2.imwrite(output_image_path, final_image)

input_image_path = '/content/drive/MyDrive/show/data/pise_sha/frame_0060.jpg'
output_image_path = '/content/drive/MyDrive/test/test_0060.jpg'
target_color = [15, 20, 16]

process_image_specific_color_range_and_blend(input_image_path, output_image_path, target_color, feather_amount=5, alpha=0.5)

output_image_path


'/content/drive/MyDrive/test/test_0060.jpg'

In [ ]:
import cv2
import numpy as np
from skimage.restoration import denoise_bilateral

# read image
image_path = '/content/drive/MyDrive/test/test_0015.jpg'
image = cv2.imread(image_path)

# Convert image from BGR to RGB
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Use bilateral filtering for noise reduction
denoised_image = denoise_bilateral(image_rgb, sigma_color=0.05, sigma_spatial=15, channel_axis=-1)

# Convert the image from RGB back to BGR
denoised_image_bgr = cv2.cvtColor((denoised_image * 255).astype(np.uint8), cv2.COLOR_RGB2BGR)

# Save the denoised image
denoised_image_path = '/content/drive/MyDrive/test/test60_s.jpg'
cv2.imwrite(denoised_image_path, denoised_image_bgr)


True

In [ ]:
# Read the denoised image
denoised_image_bgr = cv2.imread(denoised_image_path)

# Perform HD restoration
upscale_factor = 4
height, width = denoised_image_bgr.shape[:2]
upscaled_image = cv2.resize(denoised_image_bgr, (width * upscale_factor, height * upscale_factor), interpolation=cv2.INTER_CUBIC)

# Save the high-definition restored image
high_res_image_path = '/content/drive/MyDrive/test/test60_f.jpg'
cv2.imwrite(high_res_image_path, upscaled_image)

high_res_image_path


'/content/drive/MyDrive/test/test60_f.jpg'

In [ ]:
import os
import cv2
import numpy as np
from skimage.restoration import denoise_bilateral

def is_color_similar(color1, color2, threshold=30):
    return np.linalg.norm(np.array(color1) - np.array(color2)) < threshold

def adjust_brightness_contrast(image, brightness=30, contrast=-40):
    """Adjust the brightness and contrast of the image"""
    if brightness != 0:
        if brightness > 0:
            shadow = brightness
            highlight = 255
        else:
            shadow = 0
            highlight = 255 + brightness
        alpha_b = (highlight - shadow) / 255
        gamma_b = shadow
        image = cv2.addWeighted(image, alpha_b, image, 0, gamma_b)

    if contrast != 0:
        f = 131 * (contrast + 127) / (127 * (131 - contrast))
        alpha_c = f
        gamma_c = 127 * (1 - f)
        image = cv2.addWeighted(image, alpha_c, image, 0, gamma_c)

    return image

def feather_mask(mask, feather_amount=5):
    kernel_size = feather_amount * 2 + 1
    blurred_mask = cv2.GaussianBlur(mask, (kernel_size, kernel_size), 0)
    return blurred_mask

def blend_images(image1, image2, mask, alpha=0.5):
    blended = image1.copy()
    alpha_mask = mask.astype(float) / 255 * alpha
    for c in range(0, 3):
        blended[:, :, c] = (alpha_mask * image2[:, :, c] + (1 - alpha_mask) * image1[:, :, c])
    return blended

def process_image_specific_color_range_and_blend(image, target_color, color_threshold=30, brightness=35, contrast=-45, feather_amount=7, alpha=0.65):
    # Find pixels close to the target color
    mask = np.zeros(image.shape[:2], dtype=np.uint8)
    for y in range(image.shape[0]):
        for x in range(image.shape[1]):
            if is_color_similar(image[y, x], target_color, color_threshold):
                mask[y, x] = 255

    # Adjust brightness and contrast only for pixels of the target color
    adjusted_image = image.copy()
    adjusted_image[mask == 255] = adjust_brightness_contrast(image[mask == 255], brightness, contrast)

    # Feather Mask
    feathered_mask = feather_mask(mask, feather_amount)

    # Overlay the adjusted image with the original image
    final_image = blend_images(image, adjusted_image, feathered_mask, alpha)

    return final_image

def process_folder(input_folder, output_folder, target_color, color_threshold=30, brightness=35, contrast=-45, feather_amount=7, alpha=0.65, upscale_factor=4):
    # Make sure the output folder exists
    os.makedirs(output_folder, exist_ok=True)

    for filename in os.listdir(input_folder):
        input_image_path = os.path.join(input_folder, filename)
        output_image_path = os.path.join(output_folder, filename)

        # read image
        image = cv2.imread(input_image_path)
        if image is None:
            continue

        # processed image
        processed_image = process_image_specific_color_range_and_blend(image, target_color, color_threshold, brightness, contrast, feather_amount, alpha)

        # Noise reduction using bilateral filtering
        image_rgb = cv2.cvtColor(processed_image, cv2.COLOR_BGR2RGB)
        denoised_image = denoise_bilateral(image_rgb, sigma_color=0.05, sigma_spatial=15, channel_axis=-1)
        denoised_image_bgr = cv2.cvtColor((denoised_image * 255).astype(np.uint8), cv2.COLOR_RGB2BGR)

        # Performing a high-definition restoration
        height, width = denoised_image_bgr.shape[:2]
        upscaled_image = cv2.resize(denoised_image_bgr, (width * upscale_factor, height * upscale_factor), interpolation=cv2.INTER_CUBIC)

        # save
        cv2.imwrite(output_image_path, upscaled_image)


input_folder = '/content/drive/MyDrive/show/wo/input'
output_folder = '/content/drive/MyDrive/show/wo/output'
target_color = [15, 20, 16]

process_folder(input_folder, output_folder, target_color, brightness=35, contrast=-45, feather_amount=7, alpha=0.65, upscale_factor=4)

